## FINAL PROJECT


Imports

In [412]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re #regular expressions library, 
%matplotlib inline
import pandas as pd
import numpy as np
## plotting libraries
from matplotlib import pyplot as plt
import seaborn as sns
## Sklearn libraries
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline



## Dataset 1 - 'Deforestation'

Read File

In [413]:
data = pd.read_csv("TABELA_GERAL_COL7_MAPBIOMAS_DESMAT_VEGSEC_UF.csv") 
data.head()

,feature_id,city,dr_class_id,dr_class_name,class_id,level_0,level_1,level_2,level_3,level_4,...,2017,2018,2019,2020,2021,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,3,Natural,1. Forest,Forest Formation,Forest Formation,Forest Formation,...,365644.432700,363193.864600,360503.203200,358538.720700,357198.874600,NaN,NaN,NaN,NaN,NaN
1,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,4,Natural,1. Forest,Savanna Formation,Savanna Formation,Savanna Formation,...,20800.064280,20718.161970,20649.343380,20623.539650,20227.278360,NaN,NaN,NaN,NaN,NaN
2,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,11,Natural,2. Non Forest Natural Formation,Wetland,Wetland,Wetland,...,355.064644,322.257587,276.055484,373.564129,224.389942,NaN,NaN,NaN,NaN,NaN
3,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,12,Natural,2. Non Forest Natural Formation,Grassland,Grassland,Grassland,...,59188.846780,59077.497700,58898.284080,58718.878630,58970.653490,NaN,NaN,NaN,NaN,NaN
4,17694,Alta Floresta D'Oeste - RO - AMAZNIA,3,Veg. Secundária,3,Natural,1. Forest,Forest Formation,Forest Formation,Forest Formation,...,12805.204110,13579.323100,13866.430380,13966.527790,13876.363360,NaN,NaN,NaN,NaN,NaN


#### Examining Data

In [414]:
data.describe()

MemoryError: Unable to allocate 35.4 MiB for an array with shape (41, 113280) and data type float64

In [ ]:
data.info()

#### Dropping Columns

In [ ]:
data = data.drop(['Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50','Unnamed: 51'], axis = 1)
data.tail()

In [ ]:
data = data.drop(['level_0', 'level_1', 'level_2', 'level_3','level_4','color','feature_id','dr_class_id','class_id','1986','1987','1988','1989','1990','1991','1992','1993','1994','1995','1996','1997','1998'], axis = 1)
data = data.reset_index(drop=True)
data.tail()

#### Dropping Biomes


In [ ]:
data = data[:40606]
data

#### Data Cleaning

Null Values

In [ ]:
data.isnull().sum()

In [ ]:
threshold = 0.65
data = data.dropna(thresh=len(data)* (1-threshold), axis=1)
data

In [ ]:
def fill_nans_with_means(df):
    return df.fillna(df.mean()).round()

x = fill_nans_with_means(data)
x

##### Cleaning Columns 


Splitting City Column

In [ ]:
ge = x['city'].str.split(' - ', expand=True)
ge = ge.rename(columns={0: 'City', 2: 'Bioma'})
x['city'] = ge['City']
x = x.join(ge['Bioma'])
x

Renaming Columns

In [ ]:
x.rename(columns={'dr_class_name':'Deforestation', 'setor':'sector', 'municipio':'city', 'bioma':'biome', 'tipo_emissao':'type_of_emission','atividade_economica':'economic_activity'}, inplace=True)
x

#### Dropping Rows

In [ ]:
x = x[~x.isin(['Veg. Secundária', 'Supressão Veg. Primária','Recuperação para Veg. Secundária','Supressão Veg. Secundária','Outras transições']).any(axis=1)]
x

In [ ]:
len(x['Deforestation'].unique())

#### Changing values towards 'Deforestation' - Change Name 

In [ ]:
x

#### Lastly Groupby City

In [ ]:
x = x.groupby(['city','Bioma'], as_index=False).sum()
x

In [ ]:
x

## Dataset2 - 'C02 Emissions'

Read

In [ ]:
data1 = pd.read_csv("tabelao-setores-SEEG-Municipios-2_0-GWP-AR5-FINAL-SITE.csv", encoding = "ISO-8859-1") 
data1.tail()

Examining Data

In [ ]:
data1.info()

In [ ]:
data1.bioma.unique()

In [ ]:
data1

#### Dropping Biomes

In [ ]:
data1 = data1[~data1.isin(['Caatinga', 'Mata Atlântica','Pantanal','Pampa']).any(axis=1)]
data1

#### Dropping Rows

In [ ]:
data1 = data1[~data1.isin(['Energia Elétrica', 'Aluminio e Outros Não Ferrosos da Metalurgia','Ferro e Aço']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Remoção', 'Bunker']).any(axis=1)]
data1

In [ ]:
data1.nivel_2.unique()

In [ ]:
data1 = data1[~data1.isin(['Cultivo de arroz','Emissões pela Queima de Combustíveis','Efluentes Liquidos']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Produção de Combustíveis', 'Outras Industrias','Cimento','Residencial','Transporte','Metalurgia','Saneamento Básico','Uso de HFCs']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Arroz', 'Feijão','Mandioca','Milho', 'Floresta primária -- Silvicultura', 'Floresta secundária -- Silvicultura','Vegetação não florestal secundária -- Silvicultura','Uso agropecuário -- Silvicultura' ]).any(axis=1)]
data1

In [ ]:
data1.nivel_6.unique()

In [ ]:
data1

In [ ]:
data1 = data1.drop(['nivel_2', 'nivel_3', 'nivel_4', 'nivel_5','nivel_6','uf', 'ibge_cod', 'produto','setor','atividade_economica','tipo_emissao'], axis = 1)
data1.head()

Null Values

In [ ]:
data1.isnull().sum()

Renaming column

In [ ]:
data1.rename(columns={'municipio':'city'}, inplace=True)
data1

### Adding Bioma to Third Dataset Getting Prepared

In [ ]:
 data1[['city','bioma']].set_index('bioma')

In [ ]:
city_biom_dict=dict(data1[['city','bioma']].values)
city_biom_dict

In [ ]:
#city_biom_records = data1[['city','bioma']].drop_duplicates().to_dict('records')

In [ ]:
#city_biom_records

#### Lastly Groupby City

In [ ]:
#data1 = data1.groupby('city', as_index=False).sum()
#data1


## Dataset3 - 'Imports'

Import

In [ ]:
data3 = pd.read_csv("expo_combined.csv") 
data3.head()

In [ ]:
n=0

for city in data3['Município']:
    if city == city_biom_dict[n]:
        data3['Biome'] = city_biom_dict.values[0]
        n+=1
        

In [ ]:
data3

## Concatenating Datasets

Through their 'city' column

In [ ]:
data1.city.unique()

In [ ]:
x.city.unique()

In [ ]:
.city.unique()

### Cleaning Lists So That They Match

In [ ]:
import difflib

def substitute_names(list_a, list_b):
    for i, name_b in enumerate(list_b):
        for name_a in list_a:
            if difflib.SequenceMatcher(None, name_a, name_b).ratio() >= 0.7:
                list_b[i] = name_a
                break
    return list_b

substituted_list = substitute_names(data1['city'], x['city'])
substituted_list

In [ ]:
x.city

### Reset All Indexes to the Year

In [ ]:
#data.transpose()

In [ ]:
#x.transpose()

In [ ]:
#.transpose()
    

### Final Merge 

In [ ]:
#final = pd.merge(    ,x, data1, on= 'city')
#final

## Granger Causality

fig, axes = plt.subplots(nrows=4, ncols=2, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = df[df.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(df.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df, variables = df.columns)      

from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df)

nobs = 4
df_train, df_test = df[0:-nobs], df[-nobs:]

Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (4, 8)

def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

ADF Test on each column
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

1st difference
df_differenced = df_train.diff().dropna()

ADF Test on each column of 1st Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

Second Differencing
df_differenced = df_differenced.diff().dropna()

ADF Test on each column of 2nd Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')


model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

x = model.select_order(maxlags=12)
x.summary()

model_fitted = model.fit(4)
model_fitted.summary()

from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

for col, val in zip(df.columns, out):
    print(adjust(col), ':', round(val, 2))

Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

array

fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_2d')
df_forecast

def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

df_results = invert_transformation(train, df_forecast, second_diff=True)        
df_results.loc[:, ['rgnp_forecast', 'pgnp_forecast', 'ulc_forecast', 'gdfco_forecast',
                   'gdf_forecast', 'gdfim_forecast', 'gdfcf_forecast', 'gdfce_forecast']]

fig, axes = plt.subplots(nrows=int(len(df.columns)/2), ncols=2, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(df.columns, axes.flatten())):
    df_results[col+'_forecast'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    df_test[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: rgnp')
accuracy_prod = forecast_accuracy(df_results['rgnp_forecast'].values, df_test['rgnp'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: pgnp')
accuracy_prod = forecast_accuracy(df_results['pgnp_forecast'].values, df_test['pgnp'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: ulc')
accuracy_prod = forecast_accuracy(df_results['ulc_forecast'].values, df_test['ulc'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: gdfco')
accuracy_prod = forecast_accuracy(df_results['gdfco_forecast'].values, df_test['gdfco'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: gdf')
accuracy_prod = forecast_accuracy(df_results['gdf_forecast'].values, df_test['gdf'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: gdfim')
accuracy_prod = forecast_accuracy(df_results['gdfim_forecast'].values, df_test['gdfim'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: gdfcf')
accuracy_prod = forecast_accuracy(df_results['gdfcf_forecast'].values, df_test['gdfcf'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: gdfce')
accuracy_prod = forecast_accuracy(df_results['gdfce_forecast'].values, df_test['gdfce'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))